# Part 3: Deep Learning Model with Transformers
## Author: Brady Lamson
## Date: Fall 2023
# Overview and Motivation

This portion of the project has a couple distinct goals. 

Firstly, I shall utilize the `distilibert-base-uncased` model to attempt to predict the primary and secondary types of pokemon entirely from their text descriptions.

Secondly, this notebook will function as a detailed walkthrough to fine tuning a huggingface transformer. Much of the information for this task is scattered throughout documentation and articles of varying levels of utility, so compiling all of that information into one notebook will result in something that will hopefully be useful to me and anyone who may read this. 

This walkthrough will also utilize hyperparameter search using the `optuna` library, which many articles I have read online seem to lack. I hope this will give this walkthrough a useful niche that other guides have not filled.

## Potential Limitations

Performance of this model will be sought after but is not the end goal. I fear that the dataset I am working with will put a cap on performance. Pokemon types are extremely varied, with 19 types existing in this dataset alone. On top of that, I am predicting on both primary and secondary types which turns this into a multi-label prediction problem. Thus, combinations of types become important and many combinations of types only appear once. This is a limitation that likely cannot be overcome without removing problematic rows from the training split or simply acquiring more data. 

A future improvement that is outside the scope of this project is to collect all of the pokemon descriptions from each game. There are many pokemon games, and using this would allow us to duplicate many pokemon and artifically make certain type combinations more frequent and inflate our dataset. This would also provide more descriptions to train on as they tend to be similar but not identical in every game. This is obviously not without its downsides as it would inflate the frequency of already frequent type combinations, but a variant of this plan with a bit more thought put into it may be worth considering if maximizing model performance is a priority.

# Data Loading and Preprocessing